# Network flow optimization model

## Sets & Subsets

* $N$: Set of nodes in the network, i $\in$ {0,1,…,N} 

* $A$: Set of arcs in the network, k $\in$ {0,1,…,A}

* $S$ $\subset N$: Subset of nodes that represent the beginning of the network (source), s $\in$ {0,1,…,S}

* $T$ $\subset N$: Subset of nodes that represent the end of the network (sink), t $\in$ {0,1,…,T}

## Parameters

#### Node's parameters

* $l^{n}_{i}$: Is the lower storing capacity for node ${i}$, $\forall i \in N$

* $u^{n}_{i}$: Is the upper storing capacity for node ${i}$, $\forall i \in N$

* $c^{n}_{i}$: Is the unit cost for node ${i}$, $\forall i \in N$

* $\gamma^{n}_{i}$: Is the initial amount in the network for node ${i}$, $\forall i \in S \subset N$


#### Arc's parameters

* $l^{a}_{i,j}$: Is the lower storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $u^{a}_{i,j}$: Is the upper storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $c^{a}_{i,j}$: Is the unit transportation cost from node ${i}$ to node ${j}$, $\forall (i,j) \in A$

## Decision variables

#### Node's decisions

* $\tau^{n}_{i}$: Is the amount to be stored in the node ${i}$, $\forall i \in N$

#### Arc's decisions

* $\upsilon^{a}_{i,j}$: Is the amount to be send from node ${i}$ to node ${j}$, $\forall (i,j) \in A$
